### Check GPU availability

In [1]:
!nvidia-smi

Tue Feb 19 18:46:07 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   28C    P8    17W / 250W |    589MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 23%   36C    P8    17W / 250W |     10MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### Import libraries

In [2]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import copy
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from pprint import pprint
from tqdm import tqdm_notebook
from idst_util import trivial
from idst_util import dstc2
from dstc2.dstc2_scripts import score

from plotly.graph_objs import Scatter, Layout, Histogram, Histogram2d
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

[nltk_data] Downloading package punkt to /home/is/andrei-
[nltk_data]     cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Check DSTC2 availability and retrieve data

In [3]:
trivial.print_idst()

dstc2.check()

raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets()

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!
INFO:root:+-

INFO:root:Extracting raw dev features


INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features


INFO:root:Done!


### Set device

In [4]:
logging.info("+--------------------------------+")
logging.info("|             Device             |")
logging.info("+--------------------------------+")

GPU_ID = 1
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
if str(DEVICE) == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

INFO:root:+--------------------------------+
INFO:root:|             Device             |
INFO:root:+--------------------------------+
INFO:root:Running on GPU 1


### Create vocabularies

In [5]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm_notebook(raw_X_train):
    for raw_train_turn in raw_train_dialog["turns"]:
        tokens_scores = raw_train_turn["system"] + raw_train_turn["user"]
        for token_score in tokens_scores:
            token = token_score[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

logging.info("Done!")

INFO:root:+--------------------------------+
INFO:root:|          Vocabulary            |
INFO:root:+--------------------------------+
INFO:root:Creating token_to_index, index_to_token and token_to_count dictionaries


INFO:root:Done!


### Execution configuration

In [6]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)

# NOTE: we add +2 because of null and dontcare cases
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2

METHOD_DIM = len(ontology["method"])

REQUESTED_DIM = len(ontology["requestable"])

EMBEDDING_DIM = 170
ALTERED_EMBEDDING_DIM = 300
HIDDEN_DIM = 100

NUM_EPOCHS = 100
BATCH_SIZE = 10
PATIENCE = 4

GOAL_LOSS_FUNCTION = nn.CrossEntropyLoss()
METHOD_LOSS_FUNCTION = nn.CrossEntropyLoss()
REQUESTED_LOSS_FUNCTION = nn.BCELoss()

logging.info("VOCABULARY_SIZE:\t\t\t{}".format(VOCABULARY_SIZE))

logging.info("GOAL_FOOD_DIM:\t\t\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t\t\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t\t\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t\t\t{}".format(GOAL_AREA_DIM))

logging.info("METHOD_DIM:\t\t\t\t{}".format(METHOD_DIM))

logging.info("REQUESTED_DIM:\t\t\t{}".format(REQUESTED_DIM))

logging.info("EMBEDDING_DIM:\t\t\t{}".format(EMBEDDING_DIM))
logging.info("ALTERED_EMBEDDING_DIM:\t\t{}".format(ALTERED_EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t\t\t{}".format(HIDDEN_DIM))

logging.info("NUM_EPOCHS:\t\t\t\t{}".format(NUM_EPOCHS))
logging.info("BATCH_SIZE:\t\t\t\t{}".format(BATCH_SIZE))
logging.info("PATIENCE:\t\t\t\t{}".format(PATIENCE))

logging.info("GOAL_LOSS_FUNCTION:\t\t\t{}".format(GOAL_LOSS_FUNCTION))
logging.info("METHOD_LOSS_FUNCTION:\t\t\t{}".format(METHOD_LOSS_FUNCTION))
logging.info("REQUESTED_LOSS_FUNCTION:\t\t{}".format(REQUESTED_LOSS_FUNCTION))

INFO:root:+--------------------------------+
INFO:root:|         Configuration          |
INFO:root:+--------------------------------+
INFO:root:VOCABULARY_SIZE:			897
INFO:root:GOAL_FOOD_DIM:			93
INFO:root:GOAL_PRICERANGE_DIM:			5
INFO:root:GOAL_NAME_DIM:			115
INFO:root:GOAL_AREA_DIM:			7
INFO:root:METHOD_DIM:				5
INFO:root:REQUESTED_DIM:			8
INFO:root:EMBEDDING_DIM:			170
INFO:root:ALTERED_EMBEDDING_DIM:		300
INFO:root:HIDDEN_DIM:				100
INFO:root:NUM_EPOCHS:				100
INFO:root:BATCH_SIZE:				10
INFO:root:PATIENCE:				4
INFO:root:GOAL_LOSS_FUNCTION:			CrossEntropyLoss()
INFO:root:METHOD_LOSS_FUNCTION:			CrossEntropyLoss()
INFO:root:REQUESTED_LOSS_FUNCTION:		BCELoss()


### Utilities

In [7]:
def get_index_and_score(turn, token_to_index, mode, device):
    
    indices = []
    scores = []
    
    if mode == "train": # LecTrack 4.3: Out-of-Vocabulary Words
        for system_token, system_token_score in turn["system"]:
            indices.append(token_to_index[system_token])
            scores.append(system_token_score)
        for user_token, user_token_score in turn["user"]:
            if np.random.binomial(n = 1, p = 0.1) == 1:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[user_token])
            scores.append(user_token_score)
    else:
        tokens_scores = turn["system"] + turn["user"]
        for token, score in tokens_scores:
            if token not in token_to_index:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[token])
            scores.append(score)
            
    assert len(indices) == len(scores)
    
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

# --------------------

class EarlyStopping():
    
    def __init__(self, min_delta = 0, patience = 0):
        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_value):
        if np.greater((current_value - self.min_delta), self.best):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

# --------------------
    
def get_incremental_index_and_percentage(percentage, length):
    
    incremental_index = -1
    new_percentage = None
    
    if length != 0:
        incremental_index = int(np.around(percentage * length)) - 1
        new_percentage = np.around(((incremental_index + 1) / length), decimals = 2)
    
    return incremental_index, new_percentage

# --------------------

def make_tracker(model_Goal, model_Requested, model_Method, raw_X, raw_Y, dataset, percentage = 1.0):
    
    model_Goal = model_Goal.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    percentage_points = []
    
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_Goal.hidden = model_Goal.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for turn_num, (raw_X_turn, raw_Y_turn) in enumerate(zip(raw_X_dialog["turns"], raw_Y_dialog["turns"])):

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                
                # NOTE: percentage is based on user utterance
                incremental_index, new_percentage_point = get_incremental_index_and_percentage(percentage = percentage, length = len(raw_X_turn["user"]))
                incremental_index += len(raw_X_turn["system"])
                if new_percentage_point != None:
                    percentage_points.append(new_percentage_point)
                
                goal_priceranges, goal_areas, goal_names, goal_foods = model_Goal(indices, scores)
                requesteds = model_Requested(indices, scores)
                methods = model_Method(indices, scores)
                goal_pricerange = goal_priceranges[incremental_index]
                goal_name = goal_names[incremental_index]
                goal_area = goal_areas[incremental_index]
                goal_food = goal_foods[incremental_index]
                requested = requesteds[incremental_index]
                method = methods[incremental_index]
                
                turn = {}
                turn["num"] = turn_num
                turn["goal-labels"] = {}
                turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
                
                session["turns"].append(turn)
                
            tracker_json["sessions"].append(session)
            
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json, np.around(np.mean(np.array(percentage_points)), decimals = 2)

# --------------------
    
def get_scores(tracker, dataset, ontology):
    
    scores_dict = None

    if dataset == "dstc2_train":
        scores_dict = score.compute_score(dataset = "dstc2_train", dataroot = "dstc2/dstc2_traindev/data", tracker_output = tracker, ontology = ontology)
    elif dataset == "dstc2_dev":
        scores_dict = score.compute_score(dataset = "dstc2_dev", dataroot = "dstc2/dstc2_traindev/data", tracker_output = tracker, ontology = ontology)
    else: # dataset == "dstc2_test"
        scores_dict = score.compute_score(dataset = "dstc2_test", dataroot = "dstc2/dstc2_test/data", tracker_output = tracker, ontology = ontology)
            
    return scores_dict

# --------------------

def retrieve_gold_GoalPricerange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_output_GoalPricerange(output_tensor, ontology):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    goal_pricerange_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):     
        goal_pricerange_dict[ontology_informable_pricerange[index]] = output_tensor[index + 2].item()
    return goal_pricerange_dict

# --------------------

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

def retrieve_output_GoalArea(output_tensor, ontology):
    ontology_informable_area = ontology["informable"]["area"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    goal_area_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_area_dict[ontology_informable_area[index]] = output_tensor[index + 2].item()
    return goal_area_dict

# --------------------

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_output_GoalName(output_tensor, ontology):
    ontology_informable_name = ontology["informable"]["name"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    goal_name_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_name_dict[ontology_informable_name[index]] = output_tensor[index + 2].item()
    return goal_name_dict

# --------------------

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_output_GoalFood(output_tensor, ontology):
    ontology_informable_food = ontology["informable"]["food"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    goal_food_dict["dontcare"] = output_tensor[1].item() 
    for index in range(len(output_tensor) - 2):
        goal_food_dict[ontology_informable_food[index]] = output_tensor[index + 2].item()
    return goal_food_dict

# --------------------

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    gold_requested = np.zeros(len(ontology_requestable), dtype = float)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            gold_requested[ontology_requestable.index(requested)] = 1.0
    return torch.tensor([gold_requested], dtype = torch.float, device = device)

def retrieve_output_Requested(output_tensor, ontology):
    ontology_requestable = ontology["requestable"]
    output_tensor = output_tensor.view(-1)
    requested_dict = {}
    for index in range(len(output_tensor)):
        probability_value = output_tensor[index].item()
        requested_dict[ontology_requestable[index]] = probability_value
    return requested_dict

# --------------------

def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_method = ontology["method"]
    raw_gold_method = raw_Y["method"]
    gold_method = ontology_method.index(raw_gold_method)
    return torch.tensor([gold_method], dtype = torch.long, device = device)

def retrieve_output_Method(output_tensor, ontology):
    ontology_method = ontology["method"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    for index in range(len(output_tensor)):
        method_dict[ontology_method[index]] = output_tensor[index].item()
    return method_dict

### iDST Goal Model

In [8]:
class iDSTGoalModel(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_dim, altered_embedding_dim, hidden_dim,
                 goal_pricerange_dim, goal_area_dim, goal_name_dim, goal_food_dim, device):
        super(iDSTGoalModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.goal_pricerange_dim = goal_pricerange_dim
        self.goal_area_dim = goal_area_dim
        self.goal_name_dim = goal_name_dim
        self.goal_food_dim = goal_food_dim
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim)
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1), out_features = altered_embedding_dim) # +1 for the ASR-score
        self.lstm = nn.LSTM(input_size = altered_embedding_dim, hidden_size = hidden_dim)
        self.goal_pricerange_classifier = nn.Linear(in_features = hidden_dim, out_features = goal_pricerange_dim)
        self.goal_area_classifier = nn.Linear(in_features = hidden_dim, out_features = goal_area_dim)
        self.goal_name_classifier = nn.Linear(in_features = hidden_dim, out_features = goal_name_dim)
        self.goal_food_classifier = nn.Linear(in_features = hidden_dim, out_features = goal_food_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))

    def forward(self, indices, scores):
        embeddings = self.embeddings(indices)
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        goal_pricerange_output = F.log_softmax(self.goal_pricerange_classifier(lstm_out).view(-1, self.goal_pricerange_dim), dim = 1)
        goal_area_output = F.log_softmax(self.goal_area_classifier(lstm_out).view(-1, self.goal_area_dim), dim = 1)
        goal_name_output = F.log_softmax(self.goal_name_classifier(lstm_out).view(-1, self.goal_name_dim), dim = 1)
        goal_food_output = F.log_softmax(self.goal_food_classifier(lstm_out).view(-1, self.goal_food_dim), dim = 1)
        return goal_pricerange_output, goal_area_output, goal_name_output, goal_food_output

### iDST Requested Model

In [9]:
class iDSTRequestedModel(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_dim, altered_embedding_dim, hidden_dim, requested_dim, device):
        super(iDSTRequestedModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.requested_dim = requested_dim
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim)
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1), out_features = altered_embedding_dim) # +1 for the ASR-score
        self.lstm = nn.LSTM(input_size = altered_embedding_dim, hidden_size = hidden_dim)
        self.requested_classifier = nn.Linear(in_features = hidden_dim, out_features = requested_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))

    def forward(self, indices, scores):
        embeddings = self.embeddings(indices)
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        requested_output = torch.sigmoid(self.requested_classifier(lstm_out).view(-1, self.requested_dim))
        return requested_output

### iDST Method Model

In [10]:
class iDSTMethodModel(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_dim, altered_embedding_dim, hidden_dim, method_dim, device):
        super(iDSTMethodModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.method_dim = method_dim
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim)
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1), out_features = altered_embedding_dim) # +1 for the ASR-score
        self.lstm = nn.LSTM(input_size = altered_embedding_dim, hidden_size = hidden_dim)
        self.method_classifier = nn.Linear(in_features = hidden_dim, out_features = method_dim)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))

    def forward(self, indices, scores):
        embeddings = self.embeddings(indices)
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        method_output = F.log_softmax(self.method_classifier(lstm_out).view(-1, self.method_dim), dim = 1)
        return method_output

### iDST Goal Model

In [11]:
model_Goal = iDSTGoalModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                           goal_area_dim = GOAL_AREA_DIM,
                           goal_name_dim = GOAL_NAME_DIM,
                           goal_food_dim = GOAL_FOOD_DIM,
                           device = DEVICE).to(DEVICE)
optimizer_Goal = optim.Adam(model_Goal.parameters(), lr = 1e-3, amsgrad = True) 

### iDST Requested Model

In [12]:
model_Requested = iDSTRequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                     embedding_dim = EMBEDDING_DIM,
                                     altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                     hidden_dim = HIDDEN_DIM,
                                     requested_dim = REQUESTED_DIM,
                                     device = DEVICE).to(DEVICE)
optimizer_Requested = optim.Adam(model_Requested.parameters(), lr = 1e-3, amsgrad = True)

### iDST Method Model

In [13]:
model_Method = iDSTMethodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               method_dim = METHOD_DIM,
                               device = DEVICE).to(DEVICE)
optimizer_Method = optim.Adam(model_Method.parameters(), lr = 1e-3, amsgrad = True)

### Train iDST Model

In [14]:
goal_early_stopping = EarlyStopping(patience = PATIENCE)
requested_early_stopping = EarlyStopping(patience = PATIENCE)
method_early_stopping = EarlyStopping(patience = PATIENCE)

train_indices_loader = torch.utils.data.DataLoader(np.arange(raw_X_train.shape[0]), batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(NUM_EPOCHS):
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    if not goal_early_stopping.stop_training:
        model_Goal = model_Goal.train()
    if not requested_early_stopping.stop_training:
        model_Requested = model_Requested.train()
    if not method_early_stopping.stop_training:
        model_Method = model_Method.train()
    
    for train_indices in tqdm_notebook(train_indices_loader, total = len(train_indices_loader)):
        
        if not goal_early_stopping.stop_training:
            optimizer_Goal.zero_grad()
            goal_accumulated_loss = 0
        if not requested_early_stopping.stop_training:
            optimizer_Requested.zero_grad()
            requested_accumulated_loss = 0
        if not method_early_stopping.stop_training:
            optimizer_Method.zero_grad()
            method_accumulated_loss = 0
        
        for raw_X_train_dialog, raw_Y_train_dialog in zip(raw_X_train[train_indices], raw_Y_train[train_indices]):
            
            if not goal_early_stopping.stop_training:
                model_Goal.hidden = model_Goal.init_hidden()
            if not requested_early_stopping.stop_training:
                model_Requested.hidden = model_Requested.init_hidden()
            if not method_early_stopping.stop_training:
                model_Method.hidden = model_Method.init_hidden()
                
            for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):

                indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
                
                if not goal_early_stopping.stop_training:
                    goal_pricerange_outputs, goal_area_outputs, goal_name_outputs, goal_food_outputs = model_Goal(indices, scores)
                    gold_goal_pricerange = retrieve_gold_GoalPricerange(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(goal_pricerange_outputs))
                    goal_accumulated_loss += GOAL_LOSS_FUNCTION(goal_pricerange_outputs, gold_goal_pricerange)
                    gold_goal_area = retrieve_gold_GoalArea(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(goal_area_outputs))
                    goal_accumulated_loss += GOAL_LOSS_FUNCTION(goal_area_outputs, gold_goal_area)
                    gold_goal_name = retrieve_gold_GoalName(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(goal_name_outputs))
                    goal_accumulated_loss += GOAL_LOSS_FUNCTION(goal_name_outputs, gold_goal_name)
                    gold_goal_food = retrieve_gold_GoalFood(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(goal_food_outputs))
                    goal_accumulated_loss += GOAL_LOSS_FUNCTION(goal_food_outputs, gold_goal_food)
                if not requested_early_stopping.stop_training:
                    requested_outputs = model_Requested(indices, scores)
                    gold_requested = retrieve_gold_Requested(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(requested_outputs.size(0), 1)
                    requested_accumulated_loss += REQUESTED_LOSS_FUNCTION(requested_outputs, gold_requested)
                if not method_early_stopping.stop_training:
                    method_outputs = model_Method(indices, scores)
                    gold_method = retrieve_gold_Method(raw_Y_train_turn, ontology = ontology, device = DEVICE).repeat(len(method_outputs))
                    method_accumulated_loss += METHOD_LOSS_FUNCTION(method_outputs, gold_method)
                    
        if not goal_early_stopping.stop_training:
            goal_accumulated_loss.backward()
            optimizer_Goal.step()
        if not requested_early_stopping.stop_training:
            requested_accumulated_loss.backward()
            optimizer_Requested.step()
        if not method_early_stopping.stop_training:
            method_accumulated_loss.backward()
            optimizer_Method.step()
        
    dev_tracker, _ = make_tracker(model_Goal, model_Requested, model_Method, raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
    
    dev_scores_dict = get_scores(dev_tracker, dataset = "dstc2_dev", ontology = ontology)
    
    logging.info(dev_scores_dict)
    
    current_goal_score_value = dev_scores_dict["goal_pricerange_accuracy"] + dev_scores_dict["goal_area_accuracy"] + dev_scores_dict["goal_name_accuracy"] + dev_scores_dict["goal_food_accuracy"] 
    goal_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = current_goal_score_value)
    
    current_requested_score_value = dev_scores_dict["requested_all_accuracy"]
    requested_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = current_requested_score_value)
    
    current_method_score_value = dev_scores_dict["method_accuracy"]
    method_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = current_method_score_value)
    
    if goal_early_stopping.wait == 0:
        torch.save(model_Goal.state_dict(), "model_Goal.pt")
    if requested_early_stopping.wait == 0:
        torch.save(model_Requested.state_dict(), "model_Requested.pt")
    if method_early_stopping.wait == 0:
        torch.save(model_Method.state_dict(), "model_Method.pt")
        
    if goal_early_stopping.stop_training and requested_early_stopping.stop_training and method_early_stopping.stop_training:
        break

INFO:root:Epoch	1/100


INFO:root:{'goal_food_accuracy': 0.1031957, 'goal_food_l2': 0.9286195, 'goal_pricerange_accuracy': 0.3474272, 'goal_pricerange_l2': 0.7479646, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1858112, 'goal_area_accuracy': 0.2970949, 'goal_area_l2': 0.8003528, 'goal_joint_accuracy': 0.0604797, 'goal_joint_l2': 0.9888431, 'requested_all_accuracy': 0.8200505, 'requested_all_l2': 0.3613305, 'method_accuracy': 0.8688567, 'method_l2': 0.2147554}
INFO:root:Epoch	2/100


INFO:root:{'goal_food_accuracy': 0.2223702, 'goal_food_l2': 0.8683231, 'goal_pricerange_accuracy': 0.7638612, 'goal_pricerange_l2': 0.3919706, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1849702, 'goal_area_accuracy': 0.5073375, 'goal_area_l2': 0.6886395, 'goal_joint_accuracy': 0.1558916, 'goal_joint_l2': 0.9573842, 'requested_all_accuracy': 0.8276235, 'requested_all_l2': 0.3035484, 'method_accuracy': 0.8848939, 'method_l2': 0.1873177}
INFO:root:Epoch	3/100


INFO:root:{'goal_food_accuracy': 0.2509987, 'goal_food_l2': 0.8328773, 'goal_pricerange_accuracy': 0.8579976, 'goal_pricerange_l2': 0.266768, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1850146, 'goal_area_accuracy': 0.7334531, 'goal_area_l2': 0.5002961, 'goal_joint_accuracy': 0.234098, 'goal_joint_l2': 0.9074177, 'requested_all_accuracy': 0.8420483, 'requested_all_l2': 0.2394139, 'method_accuracy': 0.8859286, 'method_l2': 0.1862747}
INFO:root:Epoch	4/100


INFO:root:{'goal_food_accuracy': 0.3225699, 'goal_food_l2': 0.7973888, 'goal_pricerange_accuracy': 0.8647786, 'goal_pricerange_l2': 0.2363279, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.185148, 'goal_area_accuracy': 0.7867625, 'goal_area_l2': 0.3808392, 'goal_joint_accuracy': 0.3086548, 'goal_joint_l2': 0.8479844, 'requested_all_accuracy': 0.8748648, 'requested_all_l2': 0.1876968, 'method_accuracy': 0.8804966, 'method_l2': 0.1928232}
INFO:root:Epoch	5/100


...

INFO:root:{'goal_food_accuracy': 0.6677763, 'goal_food_l2': 0.471574, 'goal_pricerange_accuracy': 0.8695652, 'goal_pricerange_l2': 0.1983551, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1869592, 'goal_area_accuracy': 0.8077269, 'goal_area_l2': 0.2890035, 'goal_joint_accuracy': 0.5245047, 'goal_joint_l2': 0.6339073, 'requested_all_accuracy': 0.9390552, 'requested_all_l2': 0.0930544, 'method_accuracy': 0.8799793, 'method_l2': 0.1890658}
INFO:root:Epoch	26/100


INFO:root:{'goal_food_accuracy': 0.6677763, 'goal_food_l2': 0.471574, 'goal_pricerange_accuracy': 0.8695652, 'goal_pricerange_l2': 0.1983551, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1869592, 'goal_area_accuracy': 0.8077269, 'goal_area_l2': 0.2890035, 'goal_joint_accuracy': 0.5245047, 'goal_joint_l2': 0.6339073, 'requested_all_accuracy': 0.945907, 'requested_all_l2': 0.0860346, 'method_accuracy': 0.8799793, 'method_l2': 0.1890658}
INFO:root:Epoch	27/100


INFO:root:{'goal_food_accuracy': 0.6677763, 'goal_food_l2': 0.471574, 'goal_pricerange_accuracy': 0.8695652, 'goal_pricerange_l2': 0.1983551, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1869592, 'goal_area_accuracy': 0.8077269, 'goal_area_l2': 0.2890035, 'goal_joint_accuracy': 0.5245047, 'goal_joint_l2': 0.6339073, 'requested_all_accuracy': 0.9480707, 'requested_all_l2': 0.0840901, 'method_accuracy': 0.8799793, 'method_l2': 0.1890658}
INFO:root:Epoch	28/100


INFO:root:{'goal_food_accuracy': 0.6677763, 'goal_food_l2': 0.471574, 'goal_pricerange_accuracy': 0.8695652, 'goal_pricerange_l2': 0.1983551, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1869592, 'goal_area_accuracy': 0.8077269, 'goal_area_l2': 0.2890035, 'goal_joint_accuracy': 0.5245047, 'goal_joint_l2': 0.6339073, 'requested_all_accuracy': 0.9451857, 'requested_all_l2': 0.0889177, 'method_accuracy': 0.8799793, 'method_l2': 0.1890658}
INFO:root:Epoch	29/100


INFO:root:{'goal_food_accuracy': 0.6677763, 'goal_food_l2': 0.471574, 'goal_pricerange_accuracy': 0.8695652, 'goal_pricerange_l2': 0.1983551, 'goal_name_accuracy': 0.9065421, 'goal_name_l2': 0.1869592, 'goal_area_accuracy': 0.8077269, 'goal_area_l2': 0.2890035, 'goal_joint_accuracy': 0.5245047, 'goal_joint_l2': 0.6339073, 'requested_all_accuracy': 0.9451857, 'requested_all_l2': 0.0854427, 'method_accuracy': 0.8799793, 'method_l2': 0.1890658}


### Load iDST Goal Model

In [15]:
model_Goal = iDSTGoalModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                           goal_area_dim = GOAL_AREA_DIM,
                           goal_name_dim = GOAL_NAME_DIM,
                           goal_food_dim = GOAL_FOOD_DIM,
                           device = DEVICE).to(DEVICE)
model_Goal.load_state_dict(torch.load("model_Goal.pt"))
model_Goal.eval()

iDSTGoalModel(
  (embeddings): Embedding(897, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_pricerange_classifier): Linear(in_features=100, out_features=5, bias=True)
  (goal_area_classifier): Linear(in_features=100, out_features=7, bias=True)
  (goal_name_classifier): Linear(in_features=100, out_features=115, bias=True)
  (goal_food_classifier): Linear(in_features=100, out_features=93, bias=True)
)

### Load iDST Requested Model

In [16]:
model_Requested = iDSTRequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                     embedding_dim = EMBEDDING_DIM,
                                     altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                     hidden_dim = HIDDEN_DIM,
                                     requested_dim = REQUESTED_DIM,
                                     device = DEVICE).to(DEVICE)
model_Requested.load_state_dict(torch.load("model_Requested.pt"))
model_Requested.eval()

iDSTRequestedModel(
  (embeddings): Embedding(897, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (requested_classifier): Linear(in_features=100, out_features=8, bias=True)
)

In [17]:
model_Method = iDSTMethodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               method_dim = METHOD_DIM,
                               device = DEVICE).to(DEVICE)
model_Method.load_state_dict(torch.load("model_Method.pt"))
model_Method.eval()

iDSTMethodModel(
  (embeddings): Embedding(897, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (method_classifier): Linear(in_features=100, out_features=5, bias=True)
)

### Print scores

In [18]:
dev_tracker, _ = make_tracker(model_Goal, model_Requested, model_Method, raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = 1.0)
get_scores(dev_tracker, dataset = "dstc2_dev", ontology = ontology)

{'goal_food_accuracy': 0.662783,
 'goal_food_l2': 0.4737349,
 'goal_pricerange_accuracy': 0.8675708,
 'goal_pricerange_l2': 0.2149619,
 'goal_name_accuracy': 0.9065421,
 'goal_name_l2': 0.1868438,
 'goal_area_accuracy': 0.8271938,
 'goal_area_l2': 0.2737893,
 'goal_joint_accuracy': 0.5294578,
 'goal_joint_l2': 0.6356316,
 'requested_all_accuracy': 0.9484313,
 'requested_all_l2': 0.082205,
 'method_accuracy': 0.8957579,
 'method_l2': 0.1765504}

In [19]:
test_tracker, _ = make_tracker(model_Goal, model_Requested, model_Method, raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = 1.0)
get_scores(test_tracker, dataset = "dstc2_test", ontology = ontology)

{'goal_food_accuracy': 0.7371903,
 'goal_food_l2': 0.3772683,
 'goal_pricerange_accuracy': 0.8041145,
 'goal_pricerange_l2': 0.2941711,
 'goal_name_accuracy': 0.961326,
 'goal_name_l2': 0.0774807,
 'goal_area_accuracy': 0.8241194,
 'goal_area_l2': 0.2697471,
 'goal_joint_accuracy': 0.5362783,
 'goal_joint_l2': 0.618719,
 'requested_all_accuracy': 0.9513804,
 'requested_all_l2': 0.0749265,
 'method_accuracy': 0.9158456,
 'method_l2': 0.1335665}

### Plotting

In [26]:
def frange(start, stop, step):
    i = start
    while i < stop:
        i = np.around(i, decimals = 2)
        yield i
        i += step    
    
#--------------------

def plotly_plot_incremental(goal_pricerange_accuracies, goal_area_accuracies, goal_name_accuracies, goal_food_accuracies,
                            requested_accuracies, method_accuracies, percentages, dataset):
    
    if dataset == "dstc2_train":
        dataset = "TRAIN"
    elif dataset == "dstc2_dev":
        dataset = "DEV"
    else:
        dataset = "TEST"
        
    plotly.offline.iplot({"data": [Scatter(x = percentages, y = goal_pricerange_accuracies, mode = "lines+markers", name = "{} Goal Pricerange Accuracy".format(dataset), marker = dict(color = "#1abc9c")),
                                   Scatter(x = percentages, y = goal_area_accuracies, mode = "lines+markers", name = "{} Goal Area Accuracy".format(dataset), marker = dict(color = "#3498db")),
                                   Scatter(x = percentages, y = goal_name_accuracies, mode = "lines+markers", name = "{} Goal Name Accuracy".format(dataset), marker = dict(color = "#9b59b6")),
                                   Scatter(x = percentages, y = goal_food_accuracies, mode = "lines+markers", name = "{} Goal Food Accuracy".format(dataset), marker = dict(color = "#e74c3c")),
                                   Scatter(x = percentages, y = requested_accuracies, mode = "lines+markers", name = "{} Requested Accuracy".format(dataset), marker = dict(color = "#34495e")),
                                   Scatter(x = percentages, y = method_accuracies, mode = "lines+markers", name = "{} Method Accuracy".format(dataset), marker = dict(color = "#f1c40f"))],
                            "layout": Layout(title = "<b>{} Percentage - Accuracy</b>".format(dataset),
                                             xaxis = dict(title = "<b>Percentage</b>",
                                                          dtick = 0.1,
                                                          titlefont = dict(color = "#34495e")),
                                             yaxis = dict(title = "<b>Accuracy</b>",
                                                          dtick = 0.05,
                                                          titlefont = dict(color = "#34495e")),
                                             margin = Margin(b = 150))})

In [24]:
dev_goal_pricerange_accuracies = []
dev_goal_area_accuracies = []
dev_goal_name_accuracies = []
dev_goal_food_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []
dev_percentages = []

test_goal_pricerange_accuracies = []
test_goal_area_accuracies = []
test_goal_name_accuracies = []
test_goal_food_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []
test_percentages = []

percentages = list(frange(0.1, 1.05, 0.1))
for percentage in tqdm_notebook(percentages, total = len(percentages)):
    
    dev_incremental_tracker, dev_incremental_percentage = make_tracker(model_Goal, model_Requested, model_Method, raw_X_dev, raw_Y_dev, dataset = "dstc2_dev", percentage = percentage)
    dev_percentages.append(dev_incremental_percentage)
    dev_scores_dict = get_scores(dev_incremental_tracker, dataset = "dstc2_dev", ontology = ontology)
    dev_goal_pricerange_accuracies.append(dev_scores_dict["goal_pricerange_accuracy"])
    dev_goal_area_accuracies.append(dev_scores_dict["goal_area_accuracy"])
    dev_goal_name_accuracies.append(dev_scores_dict["goal_name_accuracy"])
    dev_goal_food_accuracies.append(dev_scores_dict["goal_food_accuracy"])
    dev_requested_accuracies.append(dev_scores_dict["requested_all_accuracy"])
    dev_method_accuracies.append(dev_scores_dict["method_accuracy"])
    
    test_incremental_tracker, test_incremental_percentage = make_tracker(model_Goal, model_Requested, model_Method, raw_X_test, raw_Y_test, dataset = "dstc2_test", percentage = percentage)
    test_percentages.append(test_incremental_percentage)
    test_scores_dict = get_scores(test_incremental_tracker, dataset = "dstc2_test", ontology = ontology)
    test_goal_pricerange_accuracies.append(test_scores_dict["goal_pricerange_accuracy"])
    test_goal_area_accuracies.append(test_scores_dict["goal_area_accuracy"])
    test_goal_name_accuracies.append(test_scores_dict["goal_name_accuracy"])
    test_goal_food_accuracies.append(test_scores_dict["goal_food_accuracy"])
    test_requested_accuracies.append(test_scores_dict["requested_all_accuracy"])
    test_method_accuracies.append(test_scores_dict["method_accuracy"])

In [27]:
plotly_plot_incremental(dev_goal_pricerange_accuracies, dev_goal_area_accuracies, dev_goal_name_accuracies, dev_goal_food_accuracies,
                        dev_requested_accuracies, dev_method_accuracies, dev_percentages, "dstc2_dev")

In [28]:
plotly_plot_incremental(test_goal_pricerange_accuracies, test_goal_area_accuracies, test_goal_name_accuracies, test_goal_food_accuracies,
                        test_requested_accuracies, test_method_accuracies, test_percentages, "dstc2_test")